<a href="https://colab.research.google.com/github/hyorim-jo/HelPago/blob/feat%2F%231-data-preprocessing/notebooks/helpago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HelPago

## 데이터 다운로드 및 준비

### 코드 셀 실행 로그 정보 기록 함수

In [ ]:
import time
from datetime import datetime
from google.colab import auth
import subprocess

def log_execution():
    """
    현재 코드 셀의 실행 시작/종료 시간 및 사용자 계정 정보를 출력
    """
    # 사용자 인증 (최초 1회만 입력 필요)
    auth.authenticate_user()
    try:
        user_email = subprocess.check_output(
            'gcloud config get-value account', shell=True).decode().strip()
    except Exception as e:
        user_email = f"계정 확인 실패: {e}"

    # 시작 시간
    start_time = time.time()
    start_dt = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"실행 시작: {start_dt}")
    print(f"실행 계정: {user_email}")

    # 종료는 사용자가 호출
    return start_time

### 기본 패키지 및 모듈

In [ ]:
start_time = log_execution()

import torch
import torch.nn as nn
import numpy as np

from IPython.display import Image

import matplotlib.pyplot as plt
%matplotlib inline

end_time = time.time()
end_dt = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"실행 종료: {end_dt}")
print(f"총 실행 시간: {end_time - start_time:.2f}초")

### Seed 설정 (고정)
- 학번 뒷 세자리로 설정

In [ ]:
start_time = log_execution()

torch.manual_seed(164)

end_time = time.time()
end_dt = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"실행 종료: {end_dt}")
print(f"총 실행 시간: {end_time - start_time:.2f}초")

### Colab VM에 Google Drive 마운트

In [ ]:
start_time = log_execution()

from google.colab import drive
drive.mount('/content/drive', force_remount=True) #실행하면 강제로 리마운트

end_time = time.time()
end_dt = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"실행 종료: {end_dt}")
print(f"총 실행 시간: {end_time - start_time:.2f}초")

### 데이터 복사 및 압축 해제
- copy ko-en-aihub-v1.zip from google drive to colab vm

In [ ]:
import os
import zipfile
import shutil
from tqdm import tqdm

# 경로 설정
zip_src = "/content/drive/MyDrive/datasets/ko-en-aihub-v1.zip"
zip_dst = "/content/ko-en-aihub-v1.zip"
extract_dir = "/content/ko-en-aihub-v1"
nested_extract_dir = os.path.join(extract_dir, "img_align_ko-en-aihub-v1")

# Step 1: Drive → VM 복사 (이미 있으면 생략)
print("Step 1: 파일 복사 중...")
if os.path.exists(zip_dst):
    print(f"VM에 이미 zip 파일 존재 → 복사 생략: {zip_dst}")
else:
    print("Step 1: Drive → VM zip 복사 중...")

    file_size = os.path.getsize(zip_src)
    chunk_size = 1024 * 1024  # 1MB

    with open(zip_src, 'rb') as src_file, open(zip_dst, 'wb') as dst_file:
        with tqdm(total=file_size, unit='B', unit_scale=True, desc="복사 중") as pbar:
            while True:
                chunk = src_file.read(chunk_size)
                if not chunk:
                    break
                dst_file.write(chunk)
                pbar.update(len(chunk))

    print(f"복사 완료: {zip_dst}")


# Step 2: ko-en-aihub-v1.zip 압축 해제 → /content/ko-en-aihub-v1/ (중첩 제거)
print("Step 2: 1차 압축 해제 (중첩 제거) 중...")

with zipfile.ZipFile(zip_dst, 'r') as zip_ref:
    members = zip_ref.infolist()

    # 올바른 루트 제거를 위해 실제 파일만 필터링
    real_files = [m.filename for m in members if not m.filename.endswith('/') and not m.filename.startswith('__MACOSX') and '/._' not in m.filename]

    # 가장 긴 공통 경로 prefix 계산
    root_prefix = os.path.commonprefix(real_files)
    if not root_prefix.endswith('/'):
        root_prefix = os.path.dirname(root_prefix) + "/"

    with tqdm(total=len(real_files), desc="1차 압축 해제", unit="file") as pbar:
        for member in members:
            if member.is_dir() or member.filename.startswith('__MACOSX') or '/._' in member.filename:
                continue

            # 공통 루트 제거
            rel_path = os.path.relpath(member.filename, root_prefix)
            target_path = os.path.join(extract_dir, rel_path)

            os.makedirs(os.path.dirname(target_path), exist_ok=True)
            with zip_ref.open(member) as source, open(target_path, "wb") as target:
                shutil.copyfileobj(source, target)
            pbar.update(1)

print(f"1차 압축 해제 완료: {extract_dir}")


# Step 3: img_align_ko-en-aihub-v1.zip → img_align_ko-en-aihub-v1 폴더에 해제
print("Step 3: img_align_ko-en-aihub-v1.zip 압축 해제 중...")

img_zip_path = os.path.join(extract_dir, "img_align_cko-en-aihub-v1.zip")
if os.path.exists(img_zip_path):
    os.makedirs(nested_extract_dir, exist_ok=True)
    with zipfile.ZipFile(img_zip_path, 'r') as zip_ref:
        members = zip_ref.infolist()
        with tqdm(total=len(members), desc="2차 압축 해제", unit="file") as pbar:
            for member in members:
                if member.is_dir() or member.filename.startswith('__MACOSX') or member.filename.startswith('._'):
                    pbar.update(1)
                    continue
                filename = os.path.basename(member.filename)
                if not filename:
                    pbar.update(1)
                    continue
                target_path = os.path.join(nested_extract_dir, filename)
                with zip_ref.open(member) as source, open(target_path, "wb") as target:
                    shutil.copyfileobj(source, target)
                pbar.update(1)
    print(f"img_align_ko-en-aihub-v1.zip 압축 해제 완료 → {nested_extract_dir}")
else:
    print("img_align_ko-en-aihub-v1.zip 파일이 존재하지 않습니다.")


### CelebA 데이터셋 로드
- Colab VM에 있는 데이터에 접근
- (주의) Colab Session이 종료되면 VM에 있는 데이터도 자동 삭제됨

In [ ]:
import torchvision

start_time = log_execution()

image_path = './'
ko-en-aihub-v1_train_dataset = torchvision.datasets.ko-en-aihub-v1(image_path, split='train', target_type='attr', download=False)
ko-en-aihub-v1_valid_dataset = torchvision.datasets.ko-en-aihub-v1(image_path, split='valid', target_type='attr', download=False)
ko-en-aihub-v1_test_dataset = torchvision.datasets.ko-en-aihub-v1(image_path, split='test', target_type='attr', download=False)

print('훈련 세트:', len(ko-en-aihub-v1_train_dataset))
print('검증 세트:', len(ko-en-aihub-v1_valid_dataset))
print('테스트 세트:', len(ko-en-aihub-v1_test_dataset))

end_time = time.time()
end_dt = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"실행 종료: {end_dt}")
print(f"총 실행 시간: {end_time - start_time:.2f}초")